In [1]:
# importar  librerias y  frameworks necesarios
import pandas as pd

In [8]:
# cargue de datos
data = pd.read_csv(r'C:\Users\dioct\OneDrive\Desktop\repos\demand_forcasting_and_promotions\Data\raw\data_xyz_foods.csv') 

data. head()

,Date,ProductID,StoreID,ProductName,Quantity,Price
0,2025-08-04,16a562fb5931,14bce06b5959,Pasta and Beans and Tandoori Masala,1.45,4.9
1,2025-08-26,16a562fb5931,b3815c950a50,Pasta and Beans and Tandoori Masala,1.45,4.9
2,2024-12-20,16a562fb5931,53fff28459d3,Pasta and Beans and Tandoori Masala,2.00,7.0
3,2025-04-08,2ab97e780325,8268a65fda4b,Pork belly buns and BBQ Seasoning,1.00,7.0
4,2025-04-23,2ab97e780325,024a0bfc4484,Pork belly buns and BBQ Seasoning,1.00,7.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4602519 entries, 0 to 4602518
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Date         object 
 1   ProductID    object 
 2   StoreID      object 
 3   ProductName  object 
 4   Quantity     float64
 5   Price        float64
dtypes: float64(2), object(4)
memory usage: 210.7+ MB
